In [1]:
#import Packages
import sys
!{sys.executable} -m pip install BeautifulSoup4
!{sys.executable} -m pip install geocoder
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

In [2]:
URL = "https://www.wikizeroo.org/index.php?q=aHR0cHM6Ly9lbi53aWtpcGVkaWEub3JnL3dpa2kvTGlzdF9vZl9wb3N0YWxfY29kZXNfb2ZfQ2FuYWRhOl9N"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

In [3]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1

In [4]:
#making dataframe
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighbors = pd.DataFrame(columns=column_names)
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [5]:
# initialize your variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
print(neighbors)
neighbors.shape

    PostalCode           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M6A        North York   
5          M7A  Downtown Toronto   
6          M9A         Etobicoke   
7          M1B       Scarborough   
8          M1B       Scarborough   
9          M3B        North York   
10         M4B         East York   
11         M4B         East York   
12         M5B  Downtown Toronto   
13         M5B  Downtown Toronto   
14         M6B        North York   
15         M9B         Etobicoke   
16         M9B         Etobicoke   
17         M9B         Etobicoke   
18         M9B         Etobicoke   
19         M9B         Etobicoke   
20         M1C       Scarborough   
21         M1C       Scarborough   
22         M1C       Scarborough   
23         M3C        North York   
24         M3C        North York   
25         M4C         East York   
26         M5C  Downtown Tor

(209, 5)